In [1]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
from importlib import reload
import pandas as pd
import numpy as np
import pickle
import collections
import math
import random
import sys
import csv
import unicodedata
import requests
import re
import networkx as nx
from scipy import stats
import plotly
import kaleido

import nltk

import matplotlib.pyplot as plt
import seaborn as sns

# gensim parts
from gensim import corpora
from gensim import models

### scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

import plotly.graph_objects as go

import sddk

In [2]:
import textnet

In [3]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [4]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [6]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_lemmatized_20201211.json", "df", conf)
AGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,Epici/-ae,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...",3252,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Biographi,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,10245,"[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ..."
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,"̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...",1982,"[[αδύ, ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή,..."
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,152,"[[ῥόδον, δροσόεντα, κατάπυκνος, ἐκεῖνος, ἕρπυλ..."


In [15]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
AGT["lemmata"] = AGT["lemmatized_sentences"].apply(flat_lemmata)

AGT["lemmata_wordcount"] = AGT.apply(lambda row: len(row["lemmata"]), axis=1)
AGT["lemmata_wordcount"].sum() # previously we had 13925726, then 13713183, finally 14756899

14383627

In [42]:
def get_flat_sentences(series):
    sentences_list = [sent for doc in series.tolist() for sent in doc]
    return sentences_list

subcorpora_dict = {
    "Classical (5th - 4th c. BCE)" : AGT[(AGT["date_avr"].between(-5, -3))],
    "Roman (1st - 4th c. CE)" : AGT[(AGT["date_avr"].between(0, 4)) & (AGT["provenience"]=="pagan")],
    "Christian (1st - 4th c. CE)" : AGT[(AGT["date_avr"] < 4) & (AGT["provenience"]=="christian")],
    "Jewish" : AGT[AGT["author_id"].isin(["tlg0527", "tlg0018", "tlg0526"])],
                   
    "New Testament" : AGT[AGT["author_id"].str.startswith("tlg0031")],
    "Paul of Tarsus" : AGT[AGT["author_id"]=="tlg0031paul"],               
    "Septuagint" : AGT[AGT["author_id"]=="tlg0527"],
    "Philo of Alexandria" : AGT[AGT["author_id"]=="tlg0018"],
    "Titus Flavius Josephus" : AGT[AGT["author_id"]=="tlg0526"]}

subselections = []
for key in subcorpora_dict.keys():
    subselection = subcorpora_dict[key]
    subselection_dict = {}
    wordcount = subselection["wordcount"].sum()
    doccount = len(subselection)
    lemmata = subselection["lemmata"].tolist()
    sents = get_flat_sentences(subselection["lemmatized_sentences"])
    sents_len = len(sents)
    subselection_dict.update({"label" : key, "wordcount" : wordcount, "n_sentences": sents_len, "doccount" : doccount, "sentences" : sents, "lemmata" : lemmata})
    subselections.append(subselection_dict)
subselections_df = pd.DataFrame(subselections)
subselections_df

,label,wordcount,n_sentences,doccount,sentences,lemmata
0,Classical (5th - 4th c. BCE),3668653,280475,395,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
1,Roman (1st - 4th c. CE),8692448,723956,411,"[[γεωγραφία, σόσσιε, σενεκίων, ἱστορικός, διαφ...","[[γεωγραφία, σόσσιε, σενεκίων, ἱστορικός, διαφ..."
2,Christian (1st - 4th c. CE),2975644,291084,131,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
3,Jewish,1911674,160529,90,"[[ἄλλος, νομοθέτης, ἀκαλλώπιστος, γυμνός, νομι...","[[ἄλλος, νομοθέτης, ἀκαλλώπιστος, γυμνός, νομι..."
4,New Testament,137787,8698,27,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
5,Paul of Tarsus,24066,1632,7,"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό..."
6,Septuagint,799083,57856,55,"[[εἰμί, ποιέω, θεός, οὐρανός, γῆ], [γῆ, εἰμί, ...","[[εἰμί, ποιέω, θεός, οὐρανός, γῆ, γῆ, εἰμί, ἀό..."
7,Philo of Alexandria,643011,82966,31,"[[ἄλλος, νομοθέτης, ἀκαλλώπιστος, γυμνός, νομι...","[[ἄλλος, νομοθέτης, ἀκαλλώπιστος, γυμνός, νομι..."
8,Titus Flavius Josephus,469580,19707,4,"[[ἱστορία, συγγράφω, βούλομαι, αὐτός, ὁράω, σπ...","[[ἱστορία, συγγράφω, βούλομαι, αὐτός, ὁράω, σπ..."


# term2term & term2doc matraces with sklearn - Paul as example

In [43]:
# let's explore Pa
docs = subselections_df.at[5, "sentences"]

In [48]:
len(docs)

1632

In [47]:
lemmata_lists = subselections_df.at[5, "lemmata"]

In [49]:
len(lemmata_lists) # 7 authentic letters by Paul

7

In [45]:
print(docs[:3])

[['Παῦλος', 'δοῦλος', 'Ἰησοῦς', 'Χριστός', 'κλητός', 'ἀπόστολος', 'ἀφωρισμένος', 'εὐαγγέλιον', 'θεός', 'προεπαγγέλλομαι', 'προφήτης', 'γραφή', 'ἅγιος', 'υἱός', 'γίγνομαι', 'σπέρμα', 'Δαυίδ', 'σάρξ', 'ὁρισθέντος', 'υἱός', 'θεός', 'δύναμις', 'πνεῦμα', 'ἁγιωσύνη', 'ἀνάστασις', 'νεκρός', 'Ἰησοῦς', 'Χριστός', 'κύριος', 'λαμβάνω', 'χάρις', 'ἀποστολή', 'ὑπακοή', 'πίστις', 'ἔθνος', 'ὄνομα', 'κλητός', 'Ἰησοῦς', 'Χριστός', 'πᾶς', 'ὄντα', 'Ῥώμη', 'ἀγαπητός', 'θεός', 'κλητός', 'ἅγιος'], ['χάρις', 'εἰρήνη', 'θεός', 'πατήρ', 'κύριος', 'Ἰησοῦς', 'Χριστός'], ['πρῶτος', 'εὐχαριστέω', 'θεός', 'Ἰησοῦς', 'Χριστός', 'πᾶς', 'πίστις', 'καταγγέλλω', 'ὅλος', 'κόσμος']]


comparison with the previous version of the dataset
```
[['Παῦλος', 'δοῦλος', 'Ἰησοῦς', 'Χριστός', 'κλητός', 'ἀπόστολος', 'ἀφωρισμένος', 'εὐαγγέλιον', 'θεός', 'προεπαγγέλλομαι', 'προφήτης', 'γραφή', 'ἅγιος', 'υἱός', 'γίγνομαι', 'σπέρμα', 'Δαυίδ', 'σάρξ', 'ὁρισθέντος', 'υἱός', 'θεός', 'δύναμις', 'πνεῦμα', 'ἁγιωσύνη', 'ἀνάστασις', 'νεκρός', 'Ἰησοῦς', 'Χριστός', 'κύριος', 'λαμβάνω', 'χάρις', 'ἀποστολή', 'ὑπακοή', 'πίστις', 'ἔθνος', 'ὄνομα', 'κλητός', 'Ἰησοῦς', 'Χριστός', 'πᾶς', 'ὄντα', 'Ῥώμη', 'ἀγαπητός', 'θεός', 'κλητός', 'ἅγιος'], ['χάρις', 'εἰρήνη', 'θεός', 'πατήρ', 'κύριος', 'Ἰησοῦς'], ['πρότερος', 'εὐχαριστέω', 'θεός', 'Ἰησοῦς', 'Χριστός', 'πᾶς', 'πίστις', 'καταγγέλλω', 'ὅλοξ', 'κόσμῳ.']]
```

In [46]:
vocabulary =  list(set([word for sent in docs for word in sent]))
bow = CountVectorizer(vocabulary=vocabulary)
bow_term2doc = bow.fit_transform([" ".join(doc) for doc in docs]) ### run the model
term2term_bow = (bow_term2doc.T * bow_term2doc)

In [22]:
term2term_bow

<1952x1952 sparse matrix of type '<class 'numpy.int64'>'
	with 59873 stored elements in Compressed Sparse Column format>

In [23]:
pd.DataFrame(term2term_bow.todense(), columns=vocabulary, index=vocabulary)

,ἁρπαγμός,ἀποκαλύπτω,κτίσις,βασκαίνω,αἰώνιος,πολύς,κυριακός,ἐκκλάω,οἴομαι,πίστις,...,διδακτός,ἐπισκηνόω,ἐντολή,ἀντιστρατεύομαι,αὐγάζω,ἄπειμι,κομάω,λῃστής,συναποθνῄσκω,κρείσσων
ἁρπαγμός,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ἀποκαλύπτω,0,9,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
κτίσις,0,0,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
βασκαίνω,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
αἰώνιος,0,0,0,0,15,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ἄπειμι,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,6,0,0,0,0
κομάω,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,0,0,0
λῃστής,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
συναποθνῄσκω,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [24]:
G = nx.from_numpy_matrix(term2term_bow.todense()) # from_pandas_adjacency()
vocab_dict = dict(zip(range(len(vocabulary)), vocabulary))
G = nx.relabel_nodes(G, vocab_dict)
print(G.number_of_nodes(), G.number_of_edges())

1952 30856


In [28]:
def network_from_sentences(sentences, weight_threshold=0.1):
    vocabulary =  list(set([word for sent in sentences for word in sent]))
    bow = CountVectorizer(vocabulary=vocabulary)
    bow_term2doc = bow.fit_transform([" ".join(doc) for doc in docs]) ### run the model
    term2term_bow = (bow_term2doc.T * bow_term2doc)
    G = nx.from_numpy_matrix(term2term_bow.todense()) # from_pandas_adjacency()
    vocab_dict = dict(zip(range(len(vocabulary)), vocabulary))
    G = nx.relabel_nodes(G, vocab_dict)
    edges_to_remove = []
    for edge in G.edges:
        if edge[0] == edge[1]:
            edges_to_remove.append(edge)
    for element in edges_to_remove:
        G.remove_edge(element[0], element[1])
    total_weight = sum([int(n) for n in nx.get_edge_attributes(G, "weight").values()])
    #weights = sorted([int(n) for n in nx.get_edge_attributes(G, "weight").values()], reverse=True)
    #index_position = int(len(weights) * weight_threshold)
    #minimal_weight_value = weights[index_position]
    minimal_weight_value = 2
    edges_to_remove = []
    for edge in G.edges:
        if G[edge[0]][edge[1]]["weight"] < minimal_weight_value:
            edges_to_remove.append(edge)
    for element in edges_to_remove:
        G.remove_edge(element[0], element[1])
    edges_to_remove = []
    for edge in G.edges:
        if edge[0] == edge[1]:
            edges_to_remove.append(edge)
    for element in edges_to_remove:
        G.remove_edge(element[0], element[1])
    for (u, v) in G.edges:
        G[u][v]["norm_weight"] = round((G[u][v]["weight"] / total_weight), 5)
        G[u][v]["distance"] = round(1 / (G[u][v]["weight"]), 5)
        G[u][v]["norm_distance"] = round(1 / (G[u][v]["norm_weight"] ), 5)
    return G

In [29]:
# remove 
edges_to_remove = []
for edge in G.edges:
    if edge[0] == edge[1]:
        edges_to_remove.append(edge)
for element in edges_to_remove:
    G.remove_edge(element[0], element[1])

In [30]:
total_weight = sum([int(n) for n in nx.get_edge_attributes(G, "weight").values()])
total_weight

48496

In [31]:
weights = sorted([int(n) for n in nx.get_edge_attributes(G, "weight").values()], reverse=True)

In [32]:
weights[:10]

[148, 93, 90, 71, 66, 60, 55, 50, 49, 48]

In [33]:
weight_threshold = 0.1
index_position = int(len(weights) * weight_threshold)
minimal_weight_value = weights[index_position]

In [34]:
edges_to_remove = []
for edge in G.edges:
    if G[edge[0]][edge[1]]["weight"] < minimal_weight_value:
        edges_to_remove.append(edge)
for element in edges_to_remove:
    G.remove_edge(element[0], element[1])

In [35]:
len(G.nodes)

1952

In [36]:
len(G.edges)

3278

In [37]:
list(G.edges.data())[:20]

[('ἀποκαλύπτω', 'πίστις', {'weight': 5}),
 ('ἀποκαλύπτω', 'θεός', {'weight': 7}),
 ('κτίσις', 'θεός', {'weight': 6}),
 ('κτίσις', 'εἰμί', {'weight': 3}),
 ('κτίσις', 'ὑποτάσσω', {'weight': 4}),
 ('αἰώνιος', 'θεός', {'weight': 7}),
 ('αἰώνιος', 'ἁμαρτία', {'weight': 4}),
 ('αἰώνιος', 'σάρξ', {'weight': 3}),
 ('αἰώνιος', 'χριστός', {'weight': 6}),
 ('αἰώνιος', 'ὑπερβολή', {'weight': 4}),
 ('αἰώνιος', 'βλέπω', {'weight': 8}),
 ('αἰώνιος', 'ζωή', {'weight': 5}),
 ('αἰώνιος', 'δόξα', {'weight': 5}),
 ('πολύς', 'σπουδαῖος', {'weight': 6}),
 ('πολύς', 'παρακαλέω', {'weight': 5}),
 ('πολύς', 'γνωρίζω', {'weight': 3}),
 ('πολύς', 'πρόσωπον', {'weight': 5}),
 ('πολύς', 'εὐαγγέλιον', {'weight': 5}),
 ('πολύς', 'κλῆσις', {'weight': 3}),
 ('πολύς', 'ἁπλότης', {'weight': 3})]

In [38]:
edges_to_remove = []
for edge in G.edges:
    if edge[0] == edge[1]:
        edges_to_remove.append(edge)
for element in edges_to_remove:
    G.remove_edge(element[0], element[1])

In [39]:
for (u, v) in G.edges:
    G[u][v]["norm_weight"] = round((G[u][v]["weight"] / total_weight), 5)
    G[u][v]["distance"] = round(1 / (G[u][v]["weight"]), 5)
    G[u][v]["norm_distance"] = round(1 / (G[u][v]["norm_weight"] ), 5)

In [40]:
list(G.edges.data())[:5]

[('ἀποκαλύπτω',
  'πίστις',
  {'weight': 5,
   'norm_weight': 0.0001,
   'distance': 0.2,
   'norm_distance': 10000.0}),
 ('ἀποκαλύπτω',
  'θεός',
  {'weight': 7,
   'norm_weight': 0.00014,
   'distance': 0.14286,
   'norm_distance': 7142.85714}),
 ('κτίσις',
  'θεός',
  {'weight': 6,
   'norm_weight': 0.00012,
   'distance': 0.16667,
   'norm_distance': 8333.33333}),
 ('κτίσις',
  'εἰμί',
  {'weight': 3,
   'norm_weight': 6e-05,
   'distance': 0.33333,
   'norm_distance': 16666.66667}),
 ('κτίσις',
  'ὑποτάσσω',
  {'weight': 4,
   'norm_weight': 8e-05,
   'distance': 0.25,
   'norm_distance': 12500.0})]

In [121]:
network = G.copy(as_view=False)

In [50]:
def network_from_lemmata(lemmata_list, weight_threshold=0.1):
    '''From a list of words'''
    bigrams_list = []
    for bigram in nltk.bigrams(lemmata_list):
        if bigram[0] != bigram[1]:
            bigrams_list.append(tuple(sorted(bigram)))
    G = network_object_from_bigrams(bigrams_list, weight_threshold)
    return G

def network_from_lemmata_lists(list_of_lists, weight_threshold=0.1):
    bigrams_list = []
    for lemmata_list in list_of_lists:
        for bigram in nltk.bigrams(lemmata_list):
            if bigram[0] != bigram[1]:
                bigrams_list.append(tuple(sorted(bigram)))
    G = network_object_from_bigrams(bigrams_list, weight_threshold)
    return G

def network_object_from_bigrams(bigrams_list, weight_threshold):
    bigrams_counts = list((collections.Counter(bigrams_list)).items())
    bigrams_counts = sorted(bigrams_counts, key=lambda x: x[1], reverse=True)
    G = nx.Graph()
    G.clear()
    G.add_weighted_edges_from(np.array([(bigram_count[0][0], bigram_count[0][1],  int(bigram_count[1])) for bigram_count in bigrams_counts]))
        ### add edges attributes 
    for (u, v, wt) in G.edges.data('weight'):
        G[u][v]["weight"] = int(wt)
    total_weight = sum([int(n) for n in nx.get_edge_attributes(G, "weight").values()])
    #weights = sorted([int(n) for n in nx.get_edge_attributes(G, "weight").values()], reverse=True)
    #index_position = int(len(weights) * weight_threshold)
    # minimal_weight_value = weights[index_position]
    minimal_weight_value = 2
    edges_to_remove = []
    for edge in G.edges:
        if G[edge[0]][edge[1]]["weight"] < minimal_weight_value:
            edges_to_remove.append(edge)
    for element in edges_to_remove:
        G.remove_edge(element[0], element[1])
    for (u, v) in G.edges:
        G[u][v]["norm_weight"] = round((G[u][v]["weight"] / total_weight), 5)
        G[u][v]["distance"] = round(1 / (G[u][v]["weight"]), 5)
        G[u][v]["norm_distance"] = round(1 / (G[u][v]["norm_weight"] ), 5)
    return G

In [51]:
network_bigrams = textnet.network_from_lemmata_lists(lemmata_lists)
print(len(network_bigrams.nodes()), len(network_bigrams.edges()))

1952 1271


In [79]:
network_sentences = textnet.network_from_sentences(docs)
print(len(network_sentences.nodes()), len(network_sentences.edges()))

1952 7665


In [91]:
def get_nn(source_network, source_node, per_level=5):
    return [(source_node, n[0]) for n in sorted(source_network[source_node].items(), key=lambda edge: edge[1]['weight'], reverse=True)][:per_level]

assoc_edges = [] 
assoc_edges.extend(get_nn(network_sentences, "δικαιοσύνη"))
neighbors = [e[1] for e in assoc_edges]
for nn in neighbors:
    assoc_edges.extend(get_nn(network_sentences, nn))

In [92]:
assoc_edges

[('δικαιοσύνη', 'πίστις'),
 ('δικαιοσύνη', 'θεός'),
 ('δικαιοσύνη', 'χριστός'),
 ('δικαιοσύνη', 'νόμος'),
 ('δικαιοσύνη', 'ἁμαρτία'),
 ('πίστις', 'θεός'),
 ('πίστις', 'χριστός'),
 ('πίστις', 'νόμος'),
 ('πίστις', 'δικαιοσύνη'),
 ('πίστις', 'πᾶς'),
 ('θεός', 'χριστός'),
 ('θεός', 'εἰμί'),
 ('θεός', 'πᾶς'),
 ('θεός', 'κύριος'),
 ('θεός', 'πνεῦμα'),
 ('χριστός', 'θεός'),
 ('χριστός', 'πᾶς'),
 ('χριστός', 'κύριος'),
 ('χριστός', 'εἰμί'),
 ('χριστός', 'πίστις'),
 ('νόμος', 'εἰμί'),
 ('νόμος', 'θεός'),
 ('νόμος', 'ἁμαρτία'),
 ('νόμος', 'πίστις'),
 ('νόμος', 'χριστός'),
 ('ἁμαρτία', 'νόμος'),
 ('ἁμαρτία', 'θεός'),
 ('ἁμαρτία', 'θάνατος'),
 ('ἁμαρτία', 'εἰμί'),
 ('ἁμαρτία', 'σάρξ')]

In [94]:
assoc_network = network_sentences.copy(as_view=False)
for edge in assoc_network.edges():
    if edge not in assoc_edges:
        if (edge[1],edge[0]) not in assoc_edges:
            edges_to_remove.append(edge)
assoc_network.remove_edges_from(edges_to_remove)

In [88]:
isolates = nx.isolates(ego_network)
ego_network.remove_nodes_from([n for n in isolates])

In [95]:
textnet.draw_2d_network(ego_network)

In [97]:
def get_nn(source_network, source_node, per_level=5):
    return [(source_node, n[0]) for n in sorted(source_network[source_node].items(), key=lambda edge: edge[1]['weight'], reverse=True)][:per_level]


def construct_association_network(source_network, source_term, per_level=5):
    assoc_edges = [] 
    assoc_edges.extend(get_nn(source_network, source_term, per_level))
    neighbors = [e[1] for e in assoc_edges]
    for nn in neighbors:
        assoc_edges.extend(get_nn(network_sentences, nn))

    assoc_network = source_network.copy(as_view=False)
    for edge in assoc_network.edges():
        if edge not in assoc_edges:
            if (edge[1],edge[0]) not in assoc_edges:
                edges_to_remove.append(edge)
    assoc_network.remove_edges_from(edges_to_remove)
    isolates = nx.isolates(assoc_network)
    assoc_network.remove_nodes_from([n for n in isolates])
    return assoc_network

In [100]:
assoc_network = construct_association_network(network_sentences, "δικαιοσύνη")

In [102]:
textnet.draw_2d_network(assoc_network)

In [125]:
def construct_ego_network(source_network, term, num_of_neighbours, reduced=True):
    length, path = nx.single_source_dijkstra(source_network, term, target=None, weight="distance")
    shortest_nodes = list(length.keys())[0:num_of_neighbours+1]
    path_values_sorted = [dict_pair[1] for dict_pair in sorted(path.items(), key=lambda pair: list(length.keys()).index(pair[0]))]
    path_edges = []
    for path_to_term in path_values_sorted[1:num_of_neighbours+1]:
        path_edges.extend([tuple(sorted(bigram)) for bigram in nltk.bigrams(path_to_term)])
    shortest_edges = list(set(path_edges))
    ego_network = source_network.copy(as_view=False)
    nodes_to_remove = []
    for node in ego_network.nodes:
        if node not in shortest_nodes:
            nodes_to_remove.append(node)
    for element in nodes_to_remove:
        ego_network.remove_node(element)    
    edges_to_remove = []
    if reduced==True:
        for edge in ego_network.edges:
            if edge not in shortest_edges:
                if (edge[1],edge[0]) not in shortest_edges:
                    edges_to_remove.append(edge)
        for element in edges_to_remove:
            ego_network.remove_edge(element[0], element[1])
    return ego_network

def extract_ego_network_data(ego_network, term):
    ego_network_data_prec = sorted(list(ego_network.edges.data("weight")), key=lambda tup: int(tup[2]), reverse=True)
    ego_network_data_complete = []
    for tup in ego_network_data_prec:
      if tup[1] == term:
        ego_network_data_complete.append([tup[1], tup[0], int(tup[2]), round(1 / int(tup[2]), 5)])
      else:
        ego_network_data_complete.append([tup[0], tup[1], int(tup[2]), round(1 / int(tup[2]), 5)])
    return ego_network_data_complete

In [126]:
ego_network_bigrams = construct_ego_network(network_bigrams, "χριστός", 30, reduced=False)
ego_network_bigrams.edges.data()

EdgeDataView([('Χριστός', 'Ἰησοῦς', {'weight': 83, 'norm_weight': 0.007599340780076909, 'distance': 0.01205, 'norm_distance': 131.59036}), ('Χριστός', 'ἰησοῦ.', {'weight': 14, 'norm_weight': 0.0012818165171214063, 'distance': 0.07143, 'norm_distance': 780.14286}), ('Χριστός', 'θεός', {'weight': 11, 'norm_weight': 0.0010071415491668192, 'distance': 0.09091, 'norm_distance': 992.90909}), ('Χριστός', 'εἰμί', {'weight': 10, 'norm_weight': 0.0009155832265152902, 'distance': 0.1, 'norm_distance': 1092.2}), ('Χριστός', 'κύριος', {'weight': 5, 'norm_weight': 0.0004577916132576451, 'distance': 0.2, 'norm_distance': 2184.4}), ('Χριστός', 'πᾶς', {'weight': 5, 'norm_weight': 0.0004577916132576451, 'distance': 0.2, 'norm_distance': 2184.4}), ('Χριστός', 'δίδωμι', {'weight': 5, 'norm_weight': 0.0004577916132576451, 'distance': 0.2, 'norm_distance': 2184.4}), ('Χριστός', 'πνεῦμα', {'weight': 5, 'norm_weight': 0.0004577916132576451, 'distance': 0.2, 'norm_distance': 2184.4}), ('Χριστός', 'ἔχω', {'weig

In [127]:
ego_network_sentences = construct_ego_network(network_sentences, "χριστός", 30, reduced=False)
ego_network_sentences.edges.data()

EdgeDataView([('δόξα', 'ἀδελφός', {'weight': 3, 'norm_weight': 6e-05, 'distance': 0.33333, 'norm_distance': 16666.66667}), ('δόξα', 'πνεῦμα', {'weight': 6, 'norm_weight': 0.00012, 'distance': 0.16667, 'norm_distance': 8333.33333}), ('δόξα', 'υἱός', {'weight': 3, 'norm_weight': 6e-05, 'distance': 0.33333, 'norm_distance': 16666.66667}), ('δόξα', 'πίστις', {'weight': 5, 'norm_weight': 0.0001, 'distance': 0.2, 'norm_distance': 10000.0}), ('δόξα', 'ἐγείρω', {'weight': 2, 'norm_weight': 4e-05, 'distance': 0.5, 'norm_distance': 25000.0}), ('δόξα', 'πολύς', {'weight': 6, 'norm_weight': 0.00012, 'distance': 0.16667, 'norm_distance': 8333.33333}), ('δόξα', 'πατήρ', {'weight': 7, 'norm_weight': 0.00014, 'distance': 0.14286, 'norm_distance': 7142.85714}), ('δόξα', 'πᾶς', {'weight': 11, 'norm_weight': 0.00023, 'distance': 0.09091, 'norm_distance': 4347.82609}), ('δόξα', 'δικαιοσύνη', {'weight': 3, 'norm_weight': 6e-05, 'distance': 0.33333, 'norm_distance': 16666.66667}), ('δόξα', 'εἰμί', {'weight'

In [128]:
print(list(ego_network_bigrams.nodes())) 
print(list(ego_network_sentences.nodes()))

['Χριστός', 'Ἰησοῦς', 'κύριος', 'εἰμί', 'θεός', 'χάρις', 'πατήρ', 'νομός', 'πνεῦμα', 'ἰησοῦ.', 'εἰρήνη', 'ἔργον', 'αὐτός', 'οὗτος', 'δικαιοσύνη', 'πᾶς', 'οἶδα', 'υἱός', 'δύναμις', 'θέλημα', 'ἔχω', 'ἄνθρωπος', 'δίδωμι', 'σῶμα', 'εὐχαριστέω', 'γιγνώσκω', 'εἷς', 'ἐλεύθερος', 'εὐαγγέλιον', 'σάρξ', 'χριστός']
['δόξα', 'ἀδελφός', 'πνεῦμα', 'υἱός', 'πίστις', 'ἐγείρω', 'πολύς', 'πατήρ', 'δύναμις', 'πᾶς', 'νεκρός', 'δικαιοσύνη', 'εἰμί', 'ἔχω', 'οὗτος', 'χάρις', 'χριστός', 'εὐαγγέλιον', 'κύριος', 'σάρξ', 'νόμος', 'κόσμος', 'δίδωμι', 'λόγος', 'ἔθνος', 'ἅγιος', 'οἶδα', 'γίγνομαι', 'θεός', 'ἄνθρωπος', 'σῶμα']


In [129]:
# overlap between the two networks
set(ego_network_bigrams.nodes()) &  set(ego_network_sentences.nodes())

{'δίδωμι',
 'δικαιοσύνη',
 'δύναμις',
 'εἰμί',
 'εὐαγγέλιον',
 'θεός',
 'κύριος',
 'οἶδα',
 'οὗτος',
 'πατήρ',
 'πνεῦμα',
 'πᾶς',
 'σάρξ',
 'σῶμα',
 'υἱός',
 'χάρις',
 'χριστός',
 'ἄνθρωπος',
 'ἔχω'}

In [130]:
extract_ego_network_data(ego_network_bigrams, "χριστός")

[['Χριστός', 'Ἰησοῦς', 83, 0.01205],
 ['Ἰησοῦς', 'κύριος', 55, 0.01818],
 ['εἰμί', 'θεός', 20, 0.05],
 ['θεός', 'χάρις', 20, 0.05],
 ['θεός', 'πατήρ', 18, 0.05556],
 ['εἰμί', 'νομός', 15, 0.06667],
 ['θεός', 'πνεῦμα', 15, 0.06667],
 ['Χριστός', 'ἰησοῦ.', 14, 0.07143],
 ['θεός', 'εἰρήνη', 13, 0.07692],
 ['νομός', 'ἔργον', 12, 0.08333],
 ['Χριστός', 'θεός', 11, 0.09091],
 ['κύριος', 'πατήρ', 11, 0.09091],
 ['εἰμί', 'αὐτός', 11, 0.09091],
 ['εἰμί', 'οὗτος', 11, 0.09091],
 ['Χριστός', 'εἰμί', 10, 0.1],
 ['εἰμί', 'πᾶς', 10, 0.1],
 ['θεός', 'δικαιοσύνη', 10, 0.1],
 ['θεός', 'πᾶς', 10, 0.1],
 ['θεός', 'οἶδα', 10, 0.1],
 ['εἰμί', 'σῶμα', 9, 0.11111],
 ['θεός', 'υἱός', 9, 0.11111],
 ['θεός', 'δύναμις', 9, 0.11111],
 ['θεός', 'θέλημα', 9, 0.11111],
 ['θεός', 'ἔχω', 9, 0.11111],
 ['θεός', 'δίδωμι', 9, 0.11111],
 ['πᾶς', 'ἄνθρωπος', 9, 0.11111],
 ['κύριος', 'εἰμί', 8, 0.125],
 ['κύριος', 'χάρις', 8, 0.125],
 ['εἰμί', 'πνεῦμα', 8, 0.125],
 ['θεός', 'εὐχαριστέω', 8, 0.125],
 ['θεός', 'γιγνώσκω', 8, 

In [131]:
def draw_2d_network(networkx_object):
    '''take networkX object and draw it'''
    pos_2d=nx.kamada_kawai_layout(networkx_object, weight="weight_norm")
    nx.set_node_attributes(networkx_object, pos_2d, "pos_2d")
    dmin=1
    ncenter=0
    Edges = list(networkx_object.edges)
    L=len(Edges)
    labels= list(networkx_object.nodes)
    N = len(labels)
    distance_list = [float(distance[2]) for distance in list(networkx_object.edges.data("distance"))]
    weight_list = [int(float(weight[2])) for weight in list(networkx_object.edges.data("weight"))]
    for n in pos_2d:
        x,y=pos_2d[n]
        d=(x-0.5)**2+(y-0.5)**2
        if d<dmin:
            ncenter=n
            dmin=d
    p =nx.single_source_shortest_path_length(networkx_object, ncenter)
    adjc= [len(one_adjc) for one_adjc in list((nx.generate_adjlist(networkx_object)))]
    middle_node_trace = go.Scatter(
        x=[],
        y=[],
        opacity=0,
        text=weight_list,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            opacity=0
            )
        )
    for Edge in Edges:
        x0,y0 = networkx_object.nodes[Edge[0]]["pos_2d"]
        x1,y1 = networkx_object.nodes[Edge[1]]["pos_2d"]
        middle_node_trace['x'] += tuple([(x0+x1)/2])
        middle_node_trace['y'] += tuple([(y0+y1)/2])
    edge_trace1 = go.Scatter(
        x=[], y=[],
        #hoverinfo='none',
        mode='lines',
        line=dict(width=1,color="#000000"),
        )
    edge_trace2 = go.Scatter(
        x=[],y=[],
        #hoverinfo='none',
        mode='lines',
        line=dict(width=0.7,color="#404040"),
        )
    edge_trace3 = go.Scatter(
        x=[], y=[],
        #hoverinfo='none',
        mode='lines',
        line=dict(width=0.5,color="#C0C0C0"),
        )
    best_5percent_norm_weight = sorted(list(networkx_object.edges.data("norm_weight")), key=lambda x: x[2], reverse=True)[int((len(networkx_object.edges.data("norm_weight")) / 100) * 5)][2]
    best_20percent_norm_weight = sorted(list(networkx_object.edges.data("norm_weight")), key=lambda x: x[2], reverse=True)[int((len(networkx_object.edges.data("norm_weight")) / 100) * 20)][2]
    for edge in networkx_object.edges.data():
        if edge[2]["norm_weight"] >= best_5percent_norm_weight:
            x0, y0 = networkx_object.nodes[edge[0]]['pos_2d']
            x1, y1 = networkx_object.nodes[edge[1]]['pos_2d']
            edge_trace1['x'] += tuple([x0, x1, None])
            edge_trace1['y'] += tuple([y0, y1, None])
        else:
            if edge[2]["norm_weight"] >= best_20percent_norm_weight:
                x0, y0 = networkx_object.nodes[edge[0]]['pos_2d']
                x1, y1 = networkx_object.nodes[edge[1]]['pos_2d']
                edge_trace2['x'] += tuple([x0, x1, None])
                edge_trace2['y'] += tuple([y0, y1, None])
            else:
                x0, y0 = networkx_object.nodes[edge[0]]['pos_2d']
                x1, y1 = networkx_object.nodes[edge[1]]['pos_2d']
                edge_trace3['x'] += tuple([x0, x1, None])
                edge_trace3['y'] += tuple([y0, y1, None])

    node_trace = go.Scatter(
        x=[],
        y=[],
        #name=[],
        text=[],
        textposition='bottom center',
        mode='markers+text',
        hovertext=adjc,
        hoverinfo='text',
        marker=dict(
            ###showscale=True,
            showscale=False, ### change to see scale
            colorscale='Greys',
            reversescale=True,
            color=[],
            size=7,
            colorbar=dict(
                thickness=15,
                title='degree',
                xanchor='left',
                titleside='right'
                ),
            line=dict(width=1)
            )
        )

    for node in networkx_object.nodes():
        x, y = networkx_object.nodes[node]['pos_2d']
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace["text"] += tuple([node])
        ### original version: node_trace["text"] += tuple([node])

    ### Color Node Points
    for node, adjacencies in enumerate(nx.generate_adjlist(networkx_object)):
        node_trace['marker']['color'] += tuple([len(adjacencies)])
        ###node_info = ' of connections: '+str(len(adjacencies))
        ###node_trace['something'].append(node_info)

    fig = go.Figure(data=[edge_trace1, edge_trace2, edge_trace3, node_trace, middle_node_trace],
        layout=go.Layout(
            plot_bgcolor='rgba(0,0,0,0)',
            autosize=False,
            width=500,
            height=500,
            #title=file_name,
            titlefont=dict(size=16),
            showlegend=False,
            hovermode='closest',
            margin=dict(b=10,l=10,r=10, t=10),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
            ))
    return fig

def draw_3d_network(networkx_object):
    '''take networkX object and draw it in 3D'''
    Edges = list(networkx_object.edges)
    L=len(Edges)
    distance_list = [distance[2] for distance in list(networkx_object.edges.data("distance"))]
    weight_list = [int(float(weight[2])) for weight in list(networkx_object.edges.data("weight"))]
    labels= list(networkx_object.nodes)
    N = len(labels)
    adjc= [len(one_adjc) for one_adjc in list((nx.generate_adjlist(networkx_object)))] ### instead of "group"
    pos_3d=nx.spring_layout(networkx_object, weight="weight", dim=3)
    nx.set_node_attributes(networkx_object, pos_3d, "pos_3d")
    layt = [list(array) for array in pos_3d.values()]
    N= len(networkx_object.nodes)
    Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
    Yn=[layt[k][1] for k in range(N)]# y-coordinates
    Zn=[layt[k][2] for k in range(N)]# z-coordinates
    Xe=[]
    Ye=[]
    Ze=[]
    for Edge in Edges:
        Xe+=[networkx_object.nodes[Edge[0]]["pos_3d"][0],networkx_object.nodes[Edge[1]]["pos_3d"][0], None]# x-coordinates of edge ends
        Ye+=[networkx_object.nodes[Edge[0]]["pos_3d"][1],networkx_object.nodes[Edge[1]]["pos_3d"][1], None]
        Ze+=[networkx_object.nodes[Edge[0]]["pos_3d"][2],networkx_object.nodes[Edge[1]]["pos_3d"][2], None]

        ### to get the hover into the middle of the line
        ### we have to produce a node in the middle of the line
        ### based on https://stackoverflow.com/questions/46037897/line-hover-text-in-plotly

    middle_node_trace = go.Scatter3d(
            x=[],
            y=[],
            z=[],
            opacity=0,
            text=weight_list,
            mode='markers',
            hoverinfo='text',
            marker=dict(
                opacity=0
            )
        )

    for Edge in Edges:
        x0,y0,z0 = networkx_object.nodes[Edge[0]]["pos_3d"]
        x1,y1,z1 = networkx_object.nodes[Edge[1]]["pos_3d"]
        ###trace3['x'] += [x0, x1, None]
        ###trace3['y'] += [y0, y1, None]
        ###trace3['z'] += [z0, z1, None]
        ###trace3_list.append(trace3)
        middle_node_trace['x'] += tuple([(x0+x1)/2])
        middle_node_trace['y'] += tuple([(y0+y1)/2])#.append((y0+y1)/2)
        middle_node_trace['z'] += tuple([(z0+z1)/2])#.append((z0+z1)/2)
        
    ### edge trace
    trace1=go.Scatter3d(x=Xe,
                       y=Ye,
                       z=Ze,
                       mode='lines',
                       line=dict(color='rgb(125,125,125)', width=1),
                       text=distance_list,
                       hoverinfo='text',
                       textposition="top right"
                       )
    ### node trace
    trace2=go.Scatter3d(x=Xn,
                       y=Yn,
                       z=Zn,
                       mode='markers+text',
                       ###name=labels,
                       marker=dict(symbol='circle',
                                     size=6,
                                     color=adjc,
                                     colorscale='Earth',
                                     reversescale=True,
                                     line=dict(color='rgb(50,50,50)', width=0.5)
                                     ),
                       text=[],
                       #textposition='bottom center',
                       #hovertext=adjc,
                       #hoverinfo='text'
                       )
    for node in networkx_object.nodes():
        trace2["text"] += tuple([node])
    
    axis=dict(showbackground=False,
                  showline=False,
                  zeroline=False,
                  showgrid=False,
                  showticklabels=False,
                  title=''
                  )
    layout = go.Layout(
                plot_bgcolor='rgba(0,0,0,0)',
                 title="",
                 width=900,
                 height=700,
                 showlegend=False,
                 scene=dict(
                     xaxis=dict(axis),
                     yaxis=dict(axis),
                     zaxis=dict(axis),
                ),
             margin=dict(
                t=100
            ),
            hovermode='closest',
            annotations=[
                   dict(
                   showarrow=False,
                    text="",
                    xref='paper',
                    yref='paper',
                    x=0,
                    y=0.1,
                    xanchor='left',
                    yanchor='bottom',
                    font=dict(
                    size=14
                    )
                    )
                ],    )
    data=[trace1, trace2, middle_node_trace]
    fig=go.Figure(data=data, layout=layout)
    return fig

In [132]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [133]:
ego_network = construct_ego_network(network_bigrams, "δικαιοσύνη", 30, reduced=False)
ego_network.edges.data()

EdgeDataView([('Χριστός', 'Ἰησοῦς', {'weight': 83, 'norm_weight': 0.007599340780076909, 'distance': 0.01205, 'norm_distance': 131.59036}), ('Χριστός', 'θεός', {'weight': 11, 'norm_weight': 0.0010071415491668192, 'distance': 0.09091, 'norm_distance': 992.90909}), ('Χριστός', 'εἰμί', {'weight': 10, 'norm_weight': 0.0009155832265152902, 'distance': 0.1, 'norm_distance': 1092.2}), ('Χριστός', 'κύριος', {'weight': 5, 'norm_weight': 0.0004577916132576451, 'distance': 0.2, 'norm_distance': 2184.4}), ('Χριστός', 'πᾶς', {'weight': 5, 'norm_weight': 0.0004577916132576451, 'distance': 0.2, 'norm_distance': 2184.4}), ('Χριστός', 'δίδωμι', {'weight': 5, 'norm_weight': 0.0004577916132576451, 'distance': 0.2, 'norm_distance': 2184.4}), ('Χριστός', 'πνεῦμα', {'weight': 5, 'norm_weight': 0.0004577916132576451, 'distance': 0.2, 'norm_distance': 2184.4}), ('Χριστός', 'ἔχω', {'weight': 5, 'norm_weight': 0.0004577916132576451, 'distance': 0.2, 'norm_distance': 2184.4}), ('Χριστός', 'πίστις', {'weight': 5, 

In [134]:
configure_plotly_browser_state()
ego_network = construct_ego_network(network_sentences, "δικαιοσύνη", 30, reduced=True)
ego_network.edges.data()
fig = textnet.draw_2d_network(ego_network)
fig.show()

In [140]:
fig.write_image("../figures/dik_egonet_sentences_30.png")

In [142]:
configure_plotly_browser_state()
ego_network = construct_ego_network(network_bigrams, "δικαιοσύνη", 30, reduced=True)
ego_network.edges.data()
fig = textnet.draw_2d_network(ego_network)
fig.show()

In [143]:
fig.write_image("../figures/dik_egonet_bigrams_30.png")

In [144]:
### 3d network
fig = textnet.draw_3d_network(ego_network)
fig.show()

In [ ]:
bow = CountVectorizer(vocabulary=vocabulary)
bow_term2doc = bow.fit_transform([" ".join(doc) for doc in docs]) ### run the model

term2term_bow = (bow_term2doc.T * bow_term2doc)


tfidf =  TfidfVectorizer(vocabulary=vocabulary)
tfidf_term2doc = tfidf.fit_transform([" ".join(doc) for doc in docs]) ### run the model

svd = TruncatedSVD(n_components=150, n_iter=5, random_state=42)
svd_bow = svd.fit(bow_term2doc)

svd = TruncatedSVD(n_components=150, n_iter=5, random_state=42)
svd_tfidf = svd.fit(tfidf_term2doc)
    #term2doc = pd.DataFrame(Xc.toarray(), columns=vec.get_feature_names(), index=vec.get_feature_names())
lsa_bow = pd.DataFrame(svd_bow.components_, columns=vocabulary)
lsa_tfidf = pd.DataFrame(svd_tfidf.components_, columns=vocabulary)

In [ ]:
words = ["δίκαιος", "δίκη", "δικαιοσύνη"]

In [ ]:
term2term_bow_df = pd.DataFrame(term2term_bow.todense(), columns=vocabulary, index=vocabulary)
term2term_bow_df

,πρᾶγμα,ἡσσάομαι,θλίβω,θλίψιν,διδαχῆς,ψάλλω,ἐλάχιστόν,διατάσσω,ποῦ,θερίζω,...,εἴδωλόν,φοβέω,θαυμάζω,ὀδυρμός,εὐαγγέλιον,ὑπακοην,ἀντικειμένωνἥτις,ἔνδειξις,ὑποτάσσω,συγχαίρω
πρᾶγμα,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ἡσσάομαι,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
θλίβω,0,0,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
θλίψιν,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
διδαχῆς,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ὑπακοην,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,4,0,0,0,0
ἀντικειμένωνἥτις,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,1,0,0
ἔνδειξις,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,6,0,0
ὑποτάσσω,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12,0


In [ ]:
sorted_df = term2term_bow_df.sort_values("δικαιοσύνη", ascending=False)[["δικαιοσύνη"]]
print(sorted_df.head(20))

            δικαιοσύνη
δικαιοσύνη          62
πίστις              25
θεάομαι             18
νόμος               16
λογίζομαι           11
χριστοῦ             10
ἁμαρτία             10
ἰησοῦ               10
θεός                 9
πᾶς                  9
πιστεύω              7
περιτομή             7
ἀκροβυστία           6
πατήρ                6
ἀβραάμ               6
ἀνομία               5
βασιλεύω             5
δοῦλος               5
κύριος               5
ἁμάρτημα             5


In [ ]:
sorted_df = term2term_bow_df.sort_values("δίκαιος", ascending=False)[["δίκαιος"]]
print(sorted_df.head(20))

            δίκαιος
δίκαιος          14
πίστις            8
ὅσος              6
θεός              6
νόμος             5
πᾶς               4
θεάομαι           4
ἕνος              3
δικαιοσύνη        3
ἁμός              3
ζάω               3
δικαιόω           2
ἀλλ               2
ποιέω             2
πολύς             2
ἅγιος             2
καθίστημι         2
ἔχω               2
γράφω             2
εἶμι              2


In [ ]:
lsa_model_data

,πρᾶγμα,ἡσσάομαι,θλίβω,θλίψιν,διδαχῆς,ψάλλω,ἐλάχιστόν,διατάσσω,ποῦ,θερίζω,...,εἴδωλόν,φοβέω,θαυμάζω,ὀδυρμός,εὐαγγέλιον,ὑπακοην,ἀντικειμένωνἥτις,ἔνδειξις,ὑποτάσσω,συγχαίρω
0,0.003708,0.000052,0.018253,0.000661,0.000759,0.000499,0.000129,0.001352,0.000011,0.001212,...,-6.639306e-16,0.006197,0.000482,0.000551,0.067805,0.012405,0.001671,0.005543,0.014762,0.001498
1,-0.001711,-0.000088,0.019553,-0.001290,-0.000133,-0.000006,-0.000068,-0.001622,0.000023,0.000156,...,-8.387424e-15,-0.003687,0.000134,-0.000948,-0.018718,-0.002153,0.000014,0.001900,-0.017855,-0.003475
2,-0.001938,0.000137,-0.002422,0.002927,0.000159,0.000678,0.000282,-0.000731,0.000011,0.001485,...,2.157429e-14,0.002290,0.000561,0.002798,0.067655,0.000908,0.002165,0.003588,-0.019197,-0.003370
3,-0.000222,-0.000004,-0.019747,-0.001763,0.000364,0.001681,-0.000111,0.000080,-0.000030,0.004362,...,-1.263571e-13,-0.012455,0.000969,-0.001761,-0.010202,0.017871,0.004209,0.011782,0.010564,-0.000816
4,-0.000502,-0.000153,0.002632,-0.000557,0.003583,0.000096,-0.000091,-0.001782,-0.000024,0.000647,...,3.878321e-13,0.000985,-0.000747,-0.000634,-0.038459,-0.009959,-0.001097,0.004457,-0.003471,-0.000706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,-0.010291,0.003474,-0.049004,-0.001600,-0.011729,0.012179,-0.004205,-0.002103,0.003453,-0.016910,...,-8.785520e-07,0.014278,-0.002597,-0.005485,-0.006336,0.026199,-0.013394,-0.005216,0.001714,-0.009270
146,-0.028570,0.007097,0.005474,-0.006091,0.003053,0.004164,0.001158,0.001073,-0.001872,-0.020473,...,-7.591400e-08,-0.000330,0.004361,0.010873,-0.017895,0.000099,-0.006938,-0.027157,0.002984,-0.024311
147,-0.005424,-0.004964,0.012694,-0.006861,-0.005116,0.001409,0.001862,0.009463,0.000358,0.006379,...,-5.241422e-07,0.028197,0.001656,-0.004227,-0.005124,-0.052964,0.022026,0.048630,0.046711,0.013198
148,0.021749,-0.000103,0.003712,-0.005903,-0.006872,-0.016247,0.007232,0.009815,-0.003137,-0.054888,...,-5.550533e-07,0.046293,0.007101,0.002207,-0.021809,-0.019193,-0.008599,0.001844,-0.025452,-0.006316


In [ ]:
doc2term_df = pd.DataFrame(bow_doc2term.toarray(), columns=vocabulary)
doc2term_df.head()

,πρᾶγμα,ἡσσάομαι,θλίβω,θλίψιν,διδαχῆς,ψάλλω,ἐλάχιστόν,διατάσσω,ποῦ,θερίζω,...,εἴδωλόν,φοβέω,θαυμάζω,ὀδυρμός,εὐαγγέλιον,ὑπακοην,ἀντικειμένωνἥτις,ἔνδειξις,ὑποτάσσω,συγχαίρω
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def lsa_model(docs):
    vocabulary =  list(set([word for doc in docs for word in doc]))
    tfidf =  TfidfVectorizer(vocabulary=vocabulary) ### initiaze the model
    X = tfidf.fit_transform([" ".join(doc) for doc in docs]) ### run the model
    Xc = (X.T * X)
    svd = TruncatedSVD(n_components=50, n_iter=5, random_state=42)
    svd.fit(Xc)
    #term2doc = pd.DataFrame(Xc.toarray(), columns=vec.get_feature_names(), index=vec.get_feature_names())
    lsa_model_data = pd.DataFrame(svd.components_, columns=vec.get_feature_names())
    return lsa_model_data, Xc#term2doc

In [ ]:
model, term2doc = lsa_model(christian_docs)

In [ ]:
len(model.columns)

117515

In [ ]:
def get_most_similar(model_df, target_term, number):
  all_similar = []
  for term in model_df.columns:
    similarity = (term, cosine_similarity([model_df[target_term],  model_df[term]])[0][1])
    all_similar.append(similarity)
  return sorted(all_similar, key=lambda number: number[1], reverse=True)[1:number]

In [ ]:
words = ["δίκαιος", "δίκη", "δικαιοσύνη"]

In [ ]:
neighbours = {}  
for word in words:
    neighbours[word] = get_most_similar(model, word, 10)

In [ ]:
pd.DataFrame(neighbours)

,δίκαιος,δίκη,δικαιοσύνη
0,"(ιειμένην, 0.9834371832818561)","(μετελεύσονται, 0.9384522490347709)","(καθυπηρετεῖ, 0.9710013738851778)"
1,"(εὐστοχίου, 0.9830720889621722)","(ἐπικουροί, 0.9384522490347709)","(ἐνδύσονται, 0.9706819000158255)"
2,"(λογιωτάτου, 0.9830720889621722)","(ἐριννύες, 0.9384522490347709)","(σιοί, 0.9706819000158255)"
3,"(λελύπηται, 0.9830720889621722)","(καταπεπηγμένων, 0.9344969432505092)","(περικτᾶται, 0.9699396541878471)"
4,"(τηρούσης, 0.9827599613491105)","(ἐξέτισας, 0.9182233983592916)","(ἡδυπαθείᾳ, 0.9699396541878471)"
5,"(μιμείσθω, 0.9826270670854331)","(γλωσσαργία, 0.8999555833389017)","(ἅνευ, 0.9695269024725653)"
6,"(ἰὼδ, 0.9822301096106603)","(κρυσταλλοειδεῖ, 0.8997426356399815)","(χρωννύντες, 0.9695269024725653)"
7,"(ἐπαράντων, 0.9820947343036778)","(διώκτας, 0.8970726088109655)","(πρυπορεύσεται, 0.9662484878532644)"
8,"(πειλθαρχεῖν, 0.9820947343036778)","(προςώποις, 0.8861867911971005)","(ἐπιφαινέσθω, 0.9660042345965665)"
